In [144]:
import pandas as pd
#from pandas.io.parsers import ParserError
import numpy as np
from helper import get_mapper
import json
import os
import re
from sqlalchemy import create_engine

In [145]:
from os import listdir, stat
from os.path import isfile, join
BASE_DIR = "."
MIN_SIZE = 512

In [146]:
bpm = pd.read_csv("../basic/block_plant_mapper.csv")
bpm['plantid'] = bpm['plantid'].apply(lambda x: str(x).replace('/', '_'))

In [147]:
FOLDERS = [os.path.join(BASE_DIR, o) for o in os.listdir(BASE_DIR) if os.path.isdir(os.path.join(BASE_DIR,o))]
FOLDERS.sort()
FOLDERS = FOLDERS[1:-3]

In [148]:
bpm = pd.read_csv("../basic/block_plant_mapper.csv")
bpm['plantid'] = bpm['plantid'].apply(lambda x: str(x).replace('/', '_'))
plantslist = list(set(bpm['plantid'].to_list()))

In [149]:
refpl = "03-01-01241117210.csv"

In [150]:
df = pd.read_csv("./combined/" + refpl)

In [152]:
#df

In [170]:
engine = create_engine('sqlite://', echo=False)

In [171]:
counter = 0
for plant in plantslist:
    #print(plant)
    try:
        df = pd.read_csv("./combined/" + plant + ".csv")
        df['produced_at'] =  pd.to_datetime(df['produced_at'])
        melt = df.melt(id_vars='produced_at')
        melt.to_sql('power', con=engine, if_exists='append')
        counter += 1
        #print(df.shape)
    except FileNotFoundError:
        pass
print(counter)

81


In [172]:
CDF = pd.read_sql_table('power', engine)

In [ ]:
#CDF

In [173]:
#CDF.drop(columns=['index'], inplace=True)
#CDF['produced_at'] =  pd.to_datetime(CDF['produced_at'])
CDF = CDF.set_index('produced_at') 

In [174]:
CDF["value"] = CDF["value"].astype(int)

In [177]:
#CDF

In [178]:
CDF2 = CDF.groupby('variable').resample('1M').sum()

# the inverse of groupby, reset_index
CDF3 = CDF2.reset_index()


In [179]:
CDF3

,variable,produced_at,index,value
0,BNA0015,2015-01-31,1319664,0
1,BNA0015,2015-02-28,1567668,0
2,BNA0015,2015-03-31,1975969,0
3,BNA0015,2015-04-30,2256108,0
4,BNA0015,2015-05-31,2658108,0
...,...,...,...,...
16824,BNA1949,2021-08-31,65592444,423292
16825,BNA1949,2021-09-30,63786432,544283
16826,BNA1949,2021-10-31,66364259,511561
16827,BNA1949,2021-11-30,64433136,522197


In [180]:
#gy = CDF.resample('1Y', on='produced_at').sum()
#gm = CDF.resample('1M', on='produced_at').sum()

In [181]:
gm2 = CDF3.reset_index(drop=True)
gm2['year'] = gm2['produced_at']
gm2['month'] = gm2['produced_at']

In [182]:
gm2['year'] = gm2['year'].apply(lambda x: str(x).split("-")[0])
gm2['month'] = gm2['month'].apply(lambda x: int(str(x).split("-")[1]))
gm2['month'] = gm2['month'].astype(int)

In [183]:
gm3 = gm2.sort_values(by=["year", 'month'])
gm4 = gm3.drop(columns='produced_at')

In [184]:
gm5 = gm4.reindex(columns=['year', "month", "variable", "value"])
gm6 = gm5.sort_values(by=["year", 'month'])

In [185]:
gm7 = gm6.rename(columns={"variable":"blockid", "value": "power"})
gm8 = gm7.reset_index(drop=True)

In [186]:
#gm5 = gm4.melt(id_vars=["year", "month"], var_name='blockid', value_name='power')
#gm5['power'] = gm5['power'].astype(int)

In [187]:
gm8

,year,month,blockid,power
0,2015,1,BNA0015,0
1,2015,1,BNA0016,0
2,2015,1,BNA0017,0
3,2015,1,BNA0018,0
4,2015,1,BNA0019,51476
...,...,...,...,...
16824,2021,12,BNA1673,0
16825,2021,12,BNA1817,203127
16826,2021,12,BNA1818,179694
16827,2021,12,BNA1945,75895


In [188]:
pm1 = gm7.merge(bpm, on="blockid")
pm2 = pm1.copy()

In [189]:
pm2['plantpower'] = pm2.groupby(['plantid', 'month', 'year'])['power'].transform('sum')

In [190]:
pm3 = pm2.drop_duplicates(['month', 'year', 'plantid'])
pm4 = pm3.drop(['blockid', 'power'], axis=1)

In [191]:
ym1 = pm4.copy()
ym1['yearpower'] = ym1.groupby(['plantid', 'year'])['plantpower'].transform('sum')
ym2 = ym1.drop_duplicates(['year', 'plantid'])
ym3 = ym2.drop(['month', 'plantpower'], axis=1)
ym4 = ym3.reset_index(drop=True)

In [192]:
ym4.to_csv("yearly.csv", index=False)
ym4.to_csv("yearly_pg.csv", header=False)

In [193]:
gm8.to_csv("monthly.csv", header=False)
#gm7.to_csv("yearly_pg.csv")

In [194]:
#invCDF = CDF.pivot(columns='variable')

In [195]:
ym4.loc[ym4.plantid == "06-05-100-0248923"]

,year,plantid,yearpower
342,2015,06-05-100-0248923,18265919
343,2016,06-05-100-0248923,25472064
344,2017,06-05-100-0248923,27113409
345,2018,06-05-100-0248923,29467344
346,2019,06-05-100-0248923,20842499
347,2020,06-05-100-0248923,17222434
348,2021,06-05-100-0248923,20017999


In [196]:
gm

,value
produced_at,
2015-01-31,14435600
2015-02-28,11375491
2015-03-31,12013123
2015-04-30,11250837
2015-05-31,12972905
...,...
2022-08-31,3957
2022-09-30,1658
2022-10-31,3723
